In [71]:
data=pd.read_csv(r"rainfall_data.csv")

In [68]:
data.loc[data['rainfall'] > 50, 'rainfall'] = np.nan


In [69]:
nan_rows = data[data.isna().any(axis=1)]

# print the rows with NaN values
print(nan_rows)

           date  windspeed       tpw  rainfall
2387   8/9/2019  21.575483  0.042829       NaN
2388  8/10/2019  20.276800  0.043114       NaN


In [66]:
import numpy as np

# Find rows where rainfall is NaN
null_rainfall_rows = data[data['rainfall'].isna()]

for index, null_row in null_rainfall_rows.iterrows():
    # Get windspeed and tpw of null row
    ws = null_row['windspeed']
    tpw = null_row['tpw']
    
    # Calculate Euclidean distances from null row to all other rows
    distances = np.sqrt((data['windspeed'] - ws)**2 + (data['tpw'] - tpw)**2)
    
    # Find row index of closest row
    closest_row_index = distances.idxmin()
    
    # Get rainfall value of closest row
    rainfall_value = data.loc[closest_row_index, 'rainfall']
    
    # Set rainfall value of null row to rainfall value of closest row
    data.at[index, 'rainfall'] = rainfall_value
nan_rows = data[data.isna().any(axis=1)]

# print the rows with NaN values
print(nan_rows)

           date  windspeed       tpw  rainfall
2387   8/9/2019  21.575483  0.042829       NaN
2388  8/10/2019  20.276800  0.043114       NaN


In [ ]:
data

In [33]:
import pandas as pd
from scipy.spatial import distance

# read in the data


# identify rows with missing "rainfall" values
missing_rainfall = pd.isna(data["rainfall"])

# iterate over the missing rows
for i, row in data.loc[missing_rainfall].iterrows():
    # extract windspeed and tpw values
    ws, tpw = row["windspeed"], row["tpw"]
    
        # filter out rows with missing values
    filtered_data = data.dropna(subset=["windspeed", "tpw"])

    # filter out rows with infinite values
    filtered_data = filtered_data[~np.isinf(filtered_data["windspeed"]) & ~np.isinf(filtered_data["tpw"])]

    # calculate Euclidean distances between this row and all others
    distances = [distance.euclidean((ws, tpw), (filtered_data.loc[j, "windspeed"], filtered_data.loc[j, "tpw"])) for j in filtered_data.index]

    # find the row with the smallest Euclidean distance
    closest_row = filtered_data.loc[filtered_data.index[distances.index(min(distances))]]

    # assign the "rainfall" value of the closest row to this row
    data.loc[i, "rainfall"] = closest_row["rainfall"]


ValueError: array must not contain infs or NaNs

In [25]:
import numpy as np
import copy
import tensorflow as tf
from sklearn import preprocessing
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from keras import backend as K
import os
import pandas as pd
import matplotlib.pyplot as plt
from numpy.random import seed
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import skfuzzy as fuzz
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc



def discretization(data):
    
    
    num_levels = 5

    # Generate membership functions for each variable and level
    tpw_levels = np.arange(num_levels)
    tpw_mf = [fuzz.trimf(tpw_levels, [i-1, i, i+1]) for i in tpw_levels]

    rainfall_levels = np.arange(num_levels)
    rainfall_mf = [fuzz.trimf(rainfall_levels, [i-1, i, i+1]) for i in rainfall_levels]

    windspeed_levels = np.arange(num_levels)
    windspeed_mf = [fuzz.trimf(windspeed_levels, [i-1, i, i+1]) for i in windspeed_levels]

    # Apply fuzzy membership function to the data
    tpw_fuzzy_membership = [fuzz.interp_membership(tpw_levels, m, data["tpw"]) for m in tpw_mf]
    rainfall_fuzzy_membership = [fuzz.interp_membership(rainfall_levels, m, data["rainfall"]) for m in rainfall_mf]
    windspeed_fuzzy_membership = [fuzz.interp_membership(windspeed_levels, m, data["windspeed"]) for m in windspeed_mf]

    # Assign discretization levels based on the highest membership value
    tpw_discretized = np.argmax(tpw_fuzzy_membership, axis=0)
    rainfall_discretized = np.argmax(rainfall_fuzzy_membership, axis =0)
    windspeed_discretized =np.argmax(windspeed_fuzzy_membership ,axis=0)

    # Add the discretized column to the dataframe
    data["windspeed_discretized"] = windspeed_discretized
    data["tpw_discretized"] = tpw_discretized
    data["rainfall_discretized"] = rainfall_discretized 
    return data

data=discretization(data)
extreme_data=data[data["rainfall"]>50]
normal_data=data[data["rainfall"]<=50]

def weighted_loss(normal_weight=1, extreme_weight=0.1):
    def loss(y_true, y_pred):
        normal_loss = K.mean(K.square(y_true - y_pred))
        extreme_loss = K.mean(K.square(y_true - y_pred) * K.cast(y_true > 50, 'float32'))
        return K.mean(normal_weight * normal_loss + extreme_weight * extreme_loss)
    return loss

def find_best_reg_strength(train_data, reg_strengths, num_epochs, batch_size):
    best_reg_strength = None
    best_loss = float('inf')
    for reg_strength in reg_strengths:
        # Define the autoencoder architecture
        input_layer = Input(shape=(3,))
        encoded = Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(l2=reg_strength))(input_layer)
        decoded = Dense(3, activation=None)(encoded)

        # Create the autoencoder
        autoencoder = Model(input_layer, decoded)

        # Compile the autoencoder with the weighted loss function
        autoencoder.compile(optimizer='adam', loss=weighted_loss())

        # Train the model
        history = autoencoder.fit(train_data, train_data, epochs=num_epochs, batch_size=batch_size, validation_split=0.2, verbose=0)

        # Calculate the validation loss
        val_loss = np.mean(history.history['val_loss'])

        # Update the best regularization strength and loss
        if val_loss < best_loss:
            best_reg_strength = reg_strength
            best_loss = val_loss

    print('Best regularization strength:', best_reg_strength)
    return best_reg_strength
def find_roc_curve():
    data['actual_rainfall_binary'] = (data['rainfall_class'] == 'Extreme') & (data['actual_rainfall_class'] == 'Extreme')

    fpr, tpr, thresholds = roc_curve(data['actual_rainfall_binary'], data['error'])
    roc_auc = auc(fpr, tpr)
    # Calculate TNR
    tnr = 1 - fpr

    # Calculate Youden's J statistic for each threshold
    j_stat = tpr + tnr - 1

    # Find the index of the threshold that maximizes J
    best_threshold_idx = np.argmax(j_stat)

    # Get the best threshold
    best_threshold = thresholds[best_threshold_idx]
    threshold=round(best_threshold , 1)
    return threshold

# Find the best regularization strength
reg_strengths = [0.01, 0.1, 1, 10]
num_epochs = 100
batch_size = 32
train_data = normal_data[["windspeed_discretized","tpw_discretized","rainfall_discretized"]].values.astype("float32")
best_reg_strength = find_best_reg_strength(train_data, reg_strengths, num_epochs, batch_size)

# Define the autoencoder architecture with the best regularization strength
input_layer = Input(shape=(3,))
encoded = Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(l2=best_reg_strength))(input_layer)
decoded = Dense(3, activation=None)(encoded)

# Create the autoencoder
autoencoder = Model(input_layer, decoded)

# Compile the autoencoder with the weighted loss function
autoencoder.compile(optimizer='adam', loss=weighted_loss())

# Train the model with the majority class
history = autoencoder.fit(train_data, train_data, epochs=num_epochs, batch_size=batch_size, validation_split=0.2)

# Use the trained autoencoder to predict the rainfall values for all data points
test_data = data[["windspeed_discretized","tpw_discretized"]].values
predicted_data = autoencoder.predict(test_data)
data['predicted_rainfall'] = predicted_data[:, 2]

residual=data['rainfall'].max()
def calculate_error(data, residual):
    if not pd.isna(data['rainfall']):
        return abs(data['predicted_rainfall'] - data['rainfall'])
    else:
        return abs(data['predicted_rainfall'] - residual)

data['error'] = data.apply(calculate_error, residual=residual, axis=1)

# calculate error for each row
#data['error'] = data.apply(lambda row: abs(row['predicted_rainfall'] - row['rainfall']) if not pd.isna(row['rainfall']) else abs(row['predicted_rainfall'] - residual))


# Calculate the error between the predicted and actual rainfall values
#data['error'] = np.abs(data['predicted_rainfall'] - data['rainfall'])

mean_error = np.mean(data['error'])
threshold = mean_error
print(threshold)

data['rainfall_class'] = np.where(data['error'] > threshold, 'Extreme', 'Normal')
# Define the actual rainfall class based on the threshold of 11
data['actual_rainfall_class'] = np.where(data['rainfall'] >50 , 'Extreme', 'Normal')

threshold=round(find_roc_curve() , 1)
data['rainfall_class'] = np.where(data['error'] > threshold, 'Extreme', 'Normal')

# Define the actual rainfall class based on the threshold of 11
data['actual_rainfall_class'] = np.where(data['rainfall'] >50 , 'Extreme', 'Normal')


# Calculate the accuracy
num_correct = len(data[data['rainfall_class'] == data['actual_rainfall_class']])
num_total = len(data)
accuracy = num_correct / num_total * 100
print('Accuracy: {:.2f}%'.format(accuracy))


Best regularization strength: 0.01
Epoch 1/100
65/65 [==============================] - 1s 6ms/step - loss: 1.2282 - val_loss: 1.1453
Epoch 2/100
65/65 [==============================] - 0s 2ms/step - loss: 0.8012 - val_loss: 0.7585
Epoch 3/100
65/65 [==============================] - 0s 2ms/step - loss: 0.5241 - val_loss: 0.4824
Epoch 4/100
65/65 [==============================] - 0s 3ms/step - loss: 0.3334 - val_loss: 0.2944
Epoch 5/100
65/65 [==============================] - 0s 2ms/step - loss: 0.1940 - val_loss: 0.1613
Epoch 6/100
65/65 [==============================] - 0s 2ms/step - loss: 0.1071 - val_loss: 0.0871
Epoch 7/100
65/65 [==============================] - 0s 2ms/step - loss: 0.0581 - val_loss: 0.0486
Epoch 8/100
65/65 [==============================] - 0s 2ms/step - loss: 0.0357 - val_loss: 0.0322
Epoch 9/100
65/65 [==============================] - 0s 2ms/step - loss: 0.0271 - val_loss: 0.0264
Epoch 10/100
65/65 [==============================] - 0s 2ms/step - loss: 

65/65 [==============================] - 0s 3ms/step - loss: 0.0024 - val_loss: 0.0024
Epoch 83/100
65/65 [==============================] - 0s 3ms/step - loss: 0.0023 - val_loss: 0.0023
Epoch 84/100
65/65 [==============================] - 0s 2ms/step - loss: 0.0023 - val_loss: 0.0023
Epoch 85/100
65/65 [==============================] - 0s 2ms/step - loss: 0.0023 - val_loss: 0.0022
Epoch 86/100
65/65 [==============================] - 0s 2ms/step - loss: 0.0022 - val_loss: 0.0022
Epoch 87/100
65/65 [==============================] - 0s 3ms/step - loss: 0.0022 - val_loss: 0.0022
Epoch 88/100
65/65 [==============================] - 0s 2ms/step - loss: 0.0021 - val_loss: 0.0021
Epoch 89/100
65/65 [==============================] - 0s 2ms/step - loss: 0.0021 - val_loss: 0.0021
Epoch 90/100
65/65 [==============================] - 0s 2ms/step - loss: 0.0021 - val_loss: 0.0021
Epoch 91/100
65/65 [==============================] - 0s 2ms/step - loss: 0.0021 - val_loss: 0.0021
Epoch 92/100


C:\Users\shine\anaconda3\envs\environment\lib\site-packages\sklearn\metrics\_ranking.py:1015: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(


In [ ]:

# add predicted values to dataframe as a new column
df['predicted_values'] = 
residual=df['error'].mean()
# calculate error for each row
df['error'] = df.apply(lambda row: abs(row['predicted_values'] - row['actual_value']) if not pd.isna(row['actual_value']) else abs(row['predicted_values'] - residual))



In [3]:
data["rainfall"]

0        4.688589
1       11.330589
2        6.894713
3        4.613324
4        8.111635
          ...    
2557     4.467977
2558     1.572182
2559     1.819019
2560     2.198017
2561    12.713634
Name: rainfall, Length: 2562, dtype: float64

In [26]:
wrong_predictions = data[data['rainfall_class'] != data['actual_rainfall_class']]

# Print the wrong predictions
print(wrong_predictions)

Empty DataFrame
Columns: [date, windspeed, tpw, rainfall, windspeed_discretized, tpw_discretized, rainfall_discretized, predicted_rainfall, error, rainfall_class, actual_rainfall_class, actual_rainfall_binary]
Index: []


In [24]:
data.apply(lambda row:pd.isna(row['predicted_rainfall']))

KeyError: 'predicted_rainfall'